# Example notebook to visualize sensor data acquisitions

This notebook includes some simple example to import sensor acquisitions (datalogs) into pandas dataframes for analysis and processing. It is assumed that datalogs have been created using the High Speed Datalog (HSD) package.

HSD includes a firmware binary to be flashed on one of the supported boards as well as a Python SDK that is available by default in the Vespucci development environment.

The examples below have been tested on a STEVAL-MKBOXPRO board with FP-SNS-DATALOG2 v.1.2.1.

---

==> First of all, we need to install some packages that are not included in the standard Python environment in Vespucci.

In [ ]:
!pip3 install plotly

After installation is complete, please make sure to select the "Python 3.8.3 64-bit" interpreter (top right corner) and run the following cell to make sure all needed libraries are available.

**NOW IMPORTING ALL THE REQUIRED PACKAGES AND CLASSES.**

In [ ]:
import st_hsdatalog.HSD_utils.logger as logger
from st_hsdatalog.HSD.HSDatalog import HSDatalog
from ipywidgets import *
import plotly.express as px

The firsts step is importing the Python HSD module and create an hsd object.

hsd must be created by passing a folder where acquisition data has been stored.

In [ ]:
hsd_factory = HSDatalog()
acq_folder = "../../../logs/20230905_10_38_02"
hsd = hsd_factory.create_hsd(acq_folder)
hsd.enable_timestamp_recovery(True)
component_list = HSDatalog.get_all_components(hsd, only_active=True)

In [ ]:
print(component_list)

In [ ]:
log = logger.setup_applevel_logger(is_debug = False, file_name= "app_debug.log")
start_time = 0
end_time = -1
labeled = False
raw_data = False

for component in component_list:
                try:
                    df = HSDatalog.get_dataframe(hsd, component, start_time, end_time, labeled, raw_data)
                    if not (df is None or len(df) == 0):
                        for d in df:
                            log.info("\nDataFrame - Start time: {}, End time: {}\n{}".format(start_time, d.values[-1][0] ,d))
                except Exception as err:
                    log.exception(err)

In [ ]:
acc = df[0][['A_x [g]','A_y [g]','A_z [g]']]
acc

In [ ]:
acc.describe()

In [ ]:
fig = px.line(acc.head(10000))
fig